In [37]:
#!pip install optuna

In [38]:
import optuna
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
from sklearn.metrics import accuracy_score
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
def get_anchor(a):
    quoted_part = a.split("'")[1]
    rest = a.replace(f"'{quoted_part}'", '').replace("b", '').strip()

    return quoted_part, rest

In [40]:
import re
from math import inf

def parse_range(expr: str):
    expr = expr.strip().replace(" ", "")
    
    patterns = [
        (r"^=(\-?\d+(\.\d+)?)$", 'equals'),
        (r"^(>=|>)\s*(-?\d+(\.\d+)?)$", 'lower'),
        (r"^(<=|<)\s*(-?\d+(\.\d+)?)$", 'upper'),
        (r"^(-?\d+(\.\d+)?)(<=|<){1,2}(<=|<)(-?\d+(\.\d+)?)$", 'between'),
        (r"^(-?\d+(\.\d+)?)(>=|>){1,2}(>=|>)(-?\d+(\.\d+)?)$", 'reverse_between'),
    ]
    
    for pattern, kind in patterns:
        match = re.match(pattern, expr)
        if match:
            if kind == 'equals':
                num = float(match.group(1))
                return (num, num, True, True)
            elif kind == 'lower':
                op, num = match.group(1), float(match.group(2))
                return (
                    num,
                    inf,
                    op == '>=',
                    False
                )
            elif kind == 'upper':
                op, num = match.group(1), float(match.group(2))
                return (
                    -inf,
                    num,
                    False,
                    op == '<='
                )
            elif kind == 'between':
                low = float(match.group(1))
                op1 = match.group(3)
                op2 = match.group(4)
                high = float(match.group(5))
                return (
                    low,
                    high,
                    op1 == '<=',
                    op2 == '<='
                )
            elif kind == 'reverse_between':
                high = float(match.group(1))
                op1 = match.group(3)
                op2 = match.group(4)
                low = float(match.group(5))
                return (
                    low,
                    high,
                    op2 == '>=',
                    op1 == '>='
                )

    raise ValueError(f"Unrecognized format: {expr}")

In [41]:
from typing import Optional, Tuple

def intersect(
    a: Tuple[float, float, bool, bool],
    b: Tuple[float, float, bool, bool]
) -> Optional[Tuple[float, float, bool, bool]]:
    
    a_low, a_high, a_li, a_ui = a
    b_low, b_high, b_li, b_ui = b

    # Compute max of lower bounds
    if a_low > b_low:
        low, li = a_low, a_li
    elif a_low < b_low:
        low, li = b_low, b_li
    else:
        low = a_low
        li = a_li and b_li

    # Compute min of upper bounds
    if a_high < b_high:
        high, ui = a_high, a_ui
    elif a_high > b_high:
        high, ui = b_high, b_ui
    else:
        high = a_high
        ui = a_ui and b_ui

    # Check for empty intersection
    if low > high:
        return None
    if low == high and not (li and ui):
        return None

    return (low, high, li, ui)

In [42]:
def inside(val, interval):
    low, high, li, ui = interval
    if li and ui:
        return low <= val <= high
    elif li and not ui:
        return low <= val < high
    elif not li and ui:
        return low < val <= high
    else:
        return low < val < high

In [43]:
def classify_w_anchor(input, thresholds, feature_names):
    out = np.zeros(input.shape[0])
    
    for i in range(input.shape[0]):
        for j in range(len(thresholds)):
            flag = True
            out[i] = 1
            for nk,k in enumerate(feature_names):
                if k in thresholds[j]:
                    if not (inside(input[i,nk], thresholds[j][k])):
                        flag = False
                        out[i] = 0
                        break
            if flag:
                break
            else:
                flag = True
        
    return out

In [44]:
#meta parameters
train_percentage = 80
val_percentage = 20

req_names = ['req_0', 'req_1', 'req_2', 'req_3']
req_number = len(req_names)
feature_names = ['cruise speed','image resolution','illuminance','controls responsiveness','power','smoke intensity','obstacle size','obstacle distance','firm obstacle']
feature_number = len(feature_names)

training_dataset = '../datasets/dataset5000.csv'

# Load the dataset
df = pd.read_csv(training_dataset)
n_samples = df.shape[0]
print("Number of samples: ", n_samples)

#Split 80 20 the training dataset in training and validation to have more similar data
indices = np.arange(0,n_samples)
np.random.seed(1234)
indices = np.random.permutation(indices)

training_indices = indices[0:int(n_samples*train_percentage/100)]
validation_indices = indices[int(n_samples*train_percentage/100):]

training_df = df.iloc[training_indices]
validation_df = df.iloc[validation_indices]
print('Training dataset size: ', training_df.shape)
print('Validation dataset size: ', validation_df.shape)

#select the samples that have all the requirements satisfied both in training and validation
# and drop the requirements columns
all_true_training = training_df[
    (training_df['req_0'] == 1) &
    (training_df['req_1'] == 1) &
    (training_df['req_2'] == 1) &
    (training_df['req_3'] == 1)
].drop(columns=req_names)

all_true_validation = validation_df[
    (validation_df['req_0'] == 1) &
    (validation_df['req_1'] == 1) &
    (validation_df['req_2'] == 1) &
    (validation_df['req_3'] == 1)
].drop(columns=req_names)

print('Training samples with all requirements satisfied: ', all_true_training.shape)
print('Validation samples with all requirements satisfied: ', all_true_validation.shape)

#select the samples that have one specific requirement satisfied
req_true_training = {}
for r in req_names:
    req_true_training[r] = training_df[training_df[r] == 1].drop(columns=req_names)
    print('Training samples with {} satisfied: '.format(r), req_true_training[r].shape)

req_true_validation = {}
for r in req_names:
    req_true_validation[r] = validation_df[validation_df[r] == 1].drop(columns=req_names)
    print('Validation samples with {} satisfied: '.format(r), req_true_validation[r].shape)

#create a csv with the new training data and save it
training_df.to_csv('../datasets/training_dataset.csv', index=False)
validation_df.to_csv('../datasets/validation_dataset.csv', index=False)

Number of samples:  5000
Training dataset size:  (4000, 13)
Validation dataset size:  (1000, 13)
Training samples with all requirements satisfied:  (156, 9)
Validation samples with all requirements satisfied:  (49, 9)
Training samples with req_0 satisfied:  (1382, 9)
Training samples with req_1 satisfied:  (723, 9)
Training samples with req_2 satisfied:  (908, 9)
Training samples with req_3 satisfied:  (1041, 9)
Validation samples with req_0 satisfied:  (342, 9)
Validation samples with req_1 satisfied:  (172, 9)
Validation samples with req_2 satisfied:  (235, 9)
Validation samples with req_3 satisfied:  (261, 9)


In [45]:
datasets = [] #will contain the datasets as needed by the anchor library
feature_to_use = [i for i in range(feature_number)] #contains the range of features to use
true_from_anchors_df = {}

for i,r in enumerate(req_names):
    #we load the dataset in anchors
    datasets.append(\
        utils.load_csv_dataset(\
            training_dataset, feature_number+i,\
            features_to_use=feature_to_use,\
            categorical_features=None))
    
    true_from_anchors_df[r] = np.nonzero(datasets[i].labels_train)[0]
    print('Training samples with {} satisfied: '.format(r), true_from_anchors_df[r].shape)


Training samples with req_0 satisfied:  (1365,)
Training samples with req_1 satisfied:  (725,)
Training samples with req_2 satisfied:  (903,)
Training samples with req_3 satisfied:  (1029,)


In [46]:
training_dataset = '../datasets/training_dataset.csv'
validation_dataset = '../datasets/validation_dataset.csv'

In [ ]:
import xgboost as xgb
models = [] #will contain the models (one per requirement)

explainer = []

# explanations = np.zeros((req_number, all_true_training.shape[0]), dtype=object) #will contain the explanations (objects)
# exp_txt = [] #will contain the textual explanations its structure is a matrix (list of lists) where each row corresponds to a requirement 
#              #and each column corresponds to the explanation for the corresponding row in all_true_training_dataset
model0 = xgb.XGBClassifier(random_state=1234)
model0.fit(datasets[0].train, datasets[0].labels_train)
model1 = sklearn.ensemble.GradientBoostingClassifier(random_state=1234)
model1.fit(datasets[1].train, datasets[1].labels_train)
model2 = sklearn.ensemble.RandomForestClassifier(random_state=1234)
model2.fit(datasets[2].train, datasets[2].labels_train)
model3 = sklearn.ensemble.RandomForestClassifier(random_state=1234)
model3.fit(datasets[3].train, datasets[3].labels_train)
models = [model0, model1, model2, model3]
for i in range(req_number):
    print(f"{i} out of {req_number-1}")
   
    # models.append(\
    #         sklearn.ensemble.RandomForestClassifier(random_state=1234))
    #         #sklearn.ensemble.RandomForestClassifier(random_state=1234)) #interessante per risultati casi limite
    #         #sklearn.ensemble.AdaBoostClassifier(random_state=1234))
    # models[i].fit(datasets[i].train, datasets[i].labels_train)
    
    #initialize the explainer
    explainer.append(anchor_tabular.AnchorTabularExplainer(
        datasets[i].class_names, #it maps the 0 and 1 in the dataset's requirements to the class names
        datasets[i].feature_names,
        datasets[i].train,
        datasets[i].categorical_names))

0 out of 3
1 out of 3
2 out of 3
3 out of 3


In [48]:
for i, req in enumerate(req_names):
    print(f"___________Requirement {i+1}: {req}___________")
    output = models[i].predict(datasets[i].train)

    #obtain the indices of the samples that have the requirement satisfied (truly in the dataset)
    real_values_single_req = np.where(datasets[i].labels_train == 1)[0]

    if(i == 0):
        final = np.where(output == 1)[0]
        real_values = real_values_single_req
    else:
        final = np.intersect1d(final, np.where(output == 1)[0]) 
        real_values = np.intersect1d(real_values, real_values_single_req)


positively_classified = final
print(f"Number of samples with all requirements satisfied (according to model): {positively_classified.shape[0]}")

print(f"Number of samples with all requirements satisfied (real data): {real_values.shape[0]}")
#calculate false positives
f_p = positively_classified.shape[0]- np.intersect1d(real_values, positively_classified).shape[0]
print(f"Number of false positives from model: {f_p}")
#calculate the missclassified real positive
m_r_p = real_values.shape[0] - np.intersect1d(real_values, positively_classified).shape[0]
print(f"Number of missclassified real positives: {m_r_p}")

___________Requirement 1: req_0___________
___________Requirement 2: req_1___________
___________Requirement 3: req_2___________
___________Requirement 4: req_3___________
Number of samples with all requirements satisfied (according to model): 166
Number of samples with all requirements satisfied (real data): 166
Number of false positives from model: 0
Number of missclassified real positives: 0


In [49]:
for i, req in enumerate(req_names):
    print(f"___________Requirement {i+1}: {req}___________")
    output = models[i].predict(datasets[i].train)

    #obtain the indices of the samples that have the requirement satisfied (truly in the dataset)
    real_values_single_req = datasets[i].labels_train

    if(i == 0):
        final = output
        real_values = real_values_single_req
    else:
        final *= final
        real_values *= real_values_single_req

negatively_classified = np.where(final == 0)[0]
true_negative = np.where(real_values == 0)[0]

print(f"Number of samples with all requirements satisfied (according to model): {negatively_classified.shape[0]}")
print(f"Number of samples with all requirements satisfied (real data): {true_negative.shape[0]}")
#calculate false negatives
f_n = negatively_classified.shape[0]- np.intersect1d(true_negative, negatively_classified).shape[0]
print(f"Number of false negatives from model: {f_n}")
#calculate the missclassified real negative
m_r_n = true_negative.shape[0] - np.intersect1d(true_negative, negatively_classified).shape[0]
print(f"Number of missclassified real negatives: {m_r_n}")

___________Requirement 1: req_0___________
___________Requirement 2: req_1___________
___________Requirement 3: req_2___________
___________Requirement 4: req_3___________
Number of samples with all requirements satisfied (according to model): 2635
Number of samples with all requirements satisfied (real data): 3834
Number of false negatives from model: 0
Number of missclassified real negatives: 1199


In [50]:
from multiprocessing import Pool, cpu_count

# Una funzione top-level per ricostruire i modelli/spiegatori se serve
def get_explainer_model_dataset(i):
    return explainer[i], models[i], datasets[i]

# Funzione che elabora un singolo indice j (positively classified)
def process_positive_sample(j):
    p_sample = positively_classified[j]
    intersected_exp = {}
    for i in range(req_number):
        expl, model, dataset = get_explainer_model_dataset(i)
        sample = dataset.train[p_sample]
        exp = expl.explain_instance(sample, model.predict, threshold=0.99)
        print(exp.precision())
        exp = exp.names()
        for boundings in exp:
            quoted, rest = get_anchor(boundings)
            parsed = parse_range(rest)
            if quoted not in intersected_exp:
                intersected_exp[quoted] = parsed
            else:
                intersected_exp[quoted] = intersect(intersected_exp[quoted], parsed)

    return intersected_exp

# Esegui in parallelo
with Pool(processes=cpu_count()) as pool:
    explanations = pool.map(process_positive_sample, range(len(positively_classified)))



1.0
1.0
0.9915966386554622
1.0
1.0
1.0
1.0
0.9588477366255144
0.880718954248366
0.8284839203675345
0.7555762081784386
0.7931354359925789
0.8242491657397107
0.7026022304832714
1.0
0.697594501718213
0.9912280701754386
0.6397966594045026
0.9746835443037974
1.0
0.9943661971830986
1.0
0.9722222222222222
0.9901960784313726
0.9917355371900827
0.99290780141843970.991304347826087

1.0
0.9937888198757764
0.9942857142857143
0.9910714285714286
0.8736111111111111
0.9877300613496932
1.0
0.9665071770334929
1.0
0.9488817891373802
1.0
0.9018036072144289
0.8897893030794165
0.8683068017366136
0.6025236593059937
0.9941860465116279
0.5608465608465608
1.0
1.0
0.6797839506172839
0.5662650602409639
0.8168028004667445
0.9621212121212122
1.0
1.0
0.9270833333333334
1.0
0.6923818707810994
1.0
0.995049504950495
0.9916666666666667
0.9761904761904762
1.0
0.9751552795031055
1.0
0.9714285714285714
0.9522058823529411
0.9280397022332506
1.0
1.0
0.9627906976744186
0.837138508371385
1.0
1.0
0.7516908212560387
0.9929577464

In [51]:
feature_names
missing = 0
explanations_reordered = []
for exp in explanations:
    exp_reordered = {}
    for k in feature_names:
        if k in exp:
            exp_reordered[k] = exp[k]
        else:
            exp_reordered[k] = (-inf, inf, False, False)
            print(k, "missing, added: ", exp_reordered[k])
            index = explanations.index(exp)
            missing = 1
    if missing:
        print(exp_reordered)
        missing = 0
    explanations_reordered.append(exp_reordered)
# for exp in explanations:
#     exp_reordered = {}
#     for k in feature_names:
#         if k in exp:
#             exp_reordered[k] = exp[k]
#         else:
#             exp_reordered[k] = (-inf, inf, False, False)
#             print(k, "missing, added: ", exp_reordered[k])
#             index = explanations.index(exp)
#             missing = 1
#     if missing:
#         print(exp_reordered)
#         missing = 0
#     exp = exp_reordered

smoke intensity missing, added:  (-inf, inf, False, False)
{'cruise speed': (-inf, 25.21, False, True), 'image resolution': (49.8, 75.24, False, True), 'illuminance': (26.13, 75.91, False, True), 'controls responsiveness': (73.9, inf, False, False), 'power': (51.0, 76.0, False, True), 'smoke intensity': (-inf, inf, False, False), 'obstacle size': (26.74, 50.25, False, True), 'obstacle distance': (-inf, 74.78, False, True), 'firm obstacle': (1.0, 1.0, True, True)}
image resolution missing, added:  (-inf, inf, False, False)
{'cruise speed': (-inf, 25.21, False, True), 'image resolution': (-inf, inf, False, False), 'illuminance': (26.13, 75.91, False, True), 'controls responsiveness': (73.9, inf, False, False), 'power': (76.0, inf, False, False), 'smoke intensity': (-inf, 48.87, False, True), 'obstacle size': (26.74, 50.25, False, True), 'obstacle distance': (-inf, 25.34, False, True), 'firm obstacle': (1.0, 1.0, True, True)}
power missing, added:  (-inf, inf, False, False)
{'cruise speed

In [52]:
print(explanations_reordered)
print(index)
print(explanations_reordered[index])
explanations = explanations_reordered

[{'cruise speed': (25.21, 50.17, False, True), 'image resolution': (49.8, 75.24, False, True), 'illuminance': (75.91, inf, False, False), 'controls responsiveness': (26.39, 50.29, False, True), 'power': (51.0, 76.0, False, True), 'smoke intensity': (23.42, 48.87, False, True), 'obstacle size': (74.61, inf, False, False), 'obstacle distance': (74.78, inf, False, False), 'firm obstacle': (1.0, 1.0, True, True)}, {'cruise speed': (-inf, 25.21, False, True), 'image resolution': (49.8, 75.24, False, True), 'illuminance': (26.13, 75.91, False, True), 'controls responsiveness': (73.9, inf, False, False), 'power': (51.0, 76.0, False, True), 'smoke intensity': (-inf, inf, False, False), 'obstacle size': (26.74, 50.25, False, True), 'obstacle distance': (-inf, 74.78, False, True), 'firm obstacle': (1.0, 1.0, True, True)}, {'cruise speed': (-inf, 75.49, False, True), 'image resolution': (49.8, inf, False, False), 'illuminance': (50.87, inf, False, False), 'controls responsiveness': (73.9, inf, Fa

In [53]:
def vecPredictProba(models, X):
    if type(X) is list:
        X = np.array(X)

    probas = np.empty((X.shape[0], len(models)))
    for i, model in enumerate(models):
        probas[:, i] = model.predict_proba(X)[:, 1] #Returns the probability of the positive class
    return probas

In [54]:
# def objective(trial):

#     cruise_speed = trial.suggest_float("cruise_speed",0,100)
#     image_resolution = trial.suggest_float("image_resolution",0,100)
#     illuminance = trial.suggest_float("illuminance",0,100)
#     control_responsivness = trial.suggest_float("control_responsivness",0,100)
#     power = trial.suggest_float("power",0,100)
#     smoke_intensity = trial.suggest_float("smoke_intensity",0,100)
#     obstacle_size = trial.suggest_float("obstacle_size",0,100)
#     obstacle_distance = trial.suggest_float("obstacle_distance",0,100)
#     firm_obstacle = trial.suggest_int("firm_obstacle",0,1)
#     try:
#         sample = np.array([cruise_speed,image_resolution,illuminance,control_responsivness,power,smoke_intensity,obstacle_size,obstacle_distance,firm_obstacle
#                   ]).reshape(1, -1)
#         probs = vecPredictProba(models,sample)

#         return np.mean(probs)
#     except ValueError as e:
#         print(e)
#         # TODO: How to handle the exceptions appropriately?
#         raise optuna.exceptions.TrialPruned()

In [55]:
# !pip install dotenv
# !pip install psycopg2-binary

# from dotenv import load_dotenv
# load_dotenv()


In [56]:
# import os
# storage = (os.getenv("SUPABASE_URL"))


In [57]:
# study = optuna.create_study(
#         study_name="anchors_opt",
#         storage=storage,
#         load_if_exists=True,
#         direction="maximize",
#         # sampler=optuna.samplers.RandomSampler(seed=seed)
#     )

# study.optimize(objective, n_trials=1000)

In [58]:
import numpy as np

n_samples = 1000
anchors_accuracies = []
anchors_output_accuracies = []
seed = 1234
np.random.seed(seed)  # Set the seed for reproducibility
probs_avg = np.zeros((1, 4))  # Initialize average probabilities array
for j, anchor in enumerate(explanations):
    print(f"Anchor {j}: {anchor}")
    samples = []
    positives = 0
    negatives = 0

    positive_points = 0
    negative_points = 0
    for _ in range(n_samples):
        sample = np.zeros(9)  # 8 dimensions + 1 for firm_obstacle
        for i, k in enumerate(anchor):
            a = max(anchor[k][0], 0)
            b = min(anchor[k][1], 100)
            if i == 8:
                sample[i] = anchor[k][0]  # firm_obstacle as is
            else:
                sample[i] = np.random.uniform(a, b)
        samples.append(sample)
    for sample in samples:
        #Predict th eoutput of those samples
        output = np.zeros(req_number)
        for i in range(req_number):
            output[i] = models[i].predict(np.array(sample).reshape(1, -1))
        if np.all(output == 1):
            #print("Positive output!")
            positive_points += 1
        else:
            print("output: ", output)
            print("Negative output!")
            negative_points += 1

        probabilities = vecPredictProba(models, np.array(sample).reshape(1, -1))
        probs_avg += probabilities
        if np.all(probabilities > 0.5):
            positives += 1
        else:
            negatives += 1
    probs_avg /= n_samples
    print(f"Average probabilities for anchor {j}: {probs_avg}")
    anchors_accuracies.append((positives)/(positives + negatives) * 100)
    anchors_output_accuracies.append((positive_points)/(positive_points + negative_points) * 100)
    print(f"Anchor: {j}, All positive probabilities: {positives}, All Negative probabilities: {negatives}")
    print(f"Anchor Output: {j}, Positive Points: {positive_points}, Negative Points: {negative_points}")
print(anchors_accuracies)
print(anchors_output_accuracies)


Anchor 0: {'cruise speed': (25.21, 50.17, False, True), 'image resolution': (49.8, 75.24, False, True), 'illuminance': (75.91, inf, False, False), 'controls responsiveness': (26.39, 50.29, False, True), 'power': (51.0, 76.0, False, True), 'smoke intensity': (23.42, 48.87, False, True), 'obstacle size': (74.61, inf, False, False), 'obstacle distance': (74.78, inf, False, False), 'firm obstacle': (1.0, 1.0, True, True)}
output:  [1. 0. 1. 1.]
Negative output!
output:  [1. 0. 1. 1.]
Negative output!
output:  [1. 0. 0. 0.]
Negative output!
output:  [0. 0. 1. 1.]
Negative output!
output:  [1. 0. 1. 1.]
Negative output!
output:  [0. 0. 1. 1.]
Negative output!
output:  [1. 0. 1. 1.]
Negative output!
output:  [1. 0. 1. 1.]
Negative output!
output:  [1. 0. 1. 1.]
Negative output!
output:  [1. 0. 0. 1.]
Negative output!
output:  [1. 0. 1. 1.]
Negative output!
output:  [1. 0. 0. 0.]
Negative output!
output:  [1. 0. 1. 1.]
Negative output!
output:  [1. 0. 0. 0.]
Negative output!
output:  [1. 0. 1.

KeyboardInterrupt: 

In [ ]:
#mean of the accuracies
print(f"Mean accuracy of anchors: {np.mean(anchors_accuracies)}%")
print(f"Mean accuracy of anchors output: {np.mean(anchors_output_accuracies)}%")

Mean accuracy of anchors: 54.74761904761905%
Mean accuracy of anchors output: 54.74761904761905%


: 

: 

: 

: 

: 